# pySAS Short Introduction

## Learning Goals

By the end of this tutorial, we will demonstrate:

- How to select a directory for data and analysis.
- How to copy XMM data from the HEASARC archive.
- How to run the standard XMM SAS commands `cifbuild` and `odfingest` using pySAS.

## Introduction

This tutorial provides a short, basic introduction to using pySAS. It only covers how to download observation
data files and how to calibrate the data.

This tutorial is intended for those who are already familiar with SAS commands and want to use them in Python rather than the command line.

### Inputs

- The XMM ObsID, 0802710101, of the data we will process (an observation of NGC 3079).

### Outputs


### Runtime

As of {Date}, this notebook takes ~{N}s to run to completion on Fornax using the ‘Default Astrophysics' image and the ‘{name: size}’ server with NGB RAM/ NCPU.

## Imports

In [1]:
import os

import pysas
from pysas.sastask import MyTask

There is a problem with SAS_DIR in the config file!
/does/not/exist does not exist!
Please set manually to initialize SAS.
There is a problem with SAS_CCFPATH in the config file!
/does/not/exist does not exist!
Please set manually to initialize SAS.


In [2]:
pysas.__version__

'pysas - (pysas-2.2.7) [SAS-22.1.0-a8f2c2afa-20250304]'

## Global Setup

### Functions

In [3]:
# This cell will be automatically collapsed when the notebook is rendered, which helps
#  to hide large and distracting functions while keeping the notebook self-contained
#  and leaving them easily accessible to the user

### Constants

In [4]:
OBS_ID = "0802710101"

### Configuration

The only configuration we do is to set up the root directory where we will store downloaded data.

In [5]:
if os.path.exists("../../../_data"):
    ROOT_DATA_DIR = os.path.join(os.path.abspath("../../../_data"), "XMM", "")
else:
    ROOT_DATA_DIR = "XMM/"

os.makedirs(ROOT_DATA_DIR, exist_ok=True)

***

## 1. Download observation data files (ODF) and run setup tasks

When you run the cell below, the following things will happen.

1. `basic_setup` will check if `data_dir` exists, and if not it will create it.
2. Inside data_dir `basic_setup` will create a directory with the value for the ObsID (i.e. `$data_dir/0802710101/`).
3. Within the ObsID directory, `basic_setup` will create two directories:

    a. `$data_dir/0802710101/ODF` where the observation data files are kept.

    b. `$data_dir/0802710101/work` where the `ccf.cif`, `*SUM.SAS`, and output files are kept.
4. `basic_setup` will automatically transfer the data for `obsid` to `$data_dir/0802710101/ODF` from the HEASARC archive.
5. `basic_setup` will run `cifbuild` and `odfingest`.
6. `basic_setup` will then run the basic pipeline tasks `emproc`, `epproc`, and `rgsproc`. The output of these three tasks will be in the `work_dir`.

That is it! Your data is now calibrated, processed, and ready for use with all the standard SAS commands!

In [10]:
obs = pysas.ObsID(OBS_ID, ROOT_DATA_DIR)
obs.basic_setup(repo="heasarc", overwrite=False, level="ODF")



        Starting SAS session

        Data directory = /home/jovyan/projects/heasarc-tutorials/_data/XMM/

        
INFO: Downloading data from the HEASARC ... [astroquery.heasarc.core]
INFO: Downloading to /home/jovyan/projects/heasarc-tutorials/_data/XMM//heasarc-data.tar ... [astroquery.heasarc.core]
INFO: Untar /home/jovyan/projects/heasarc-tutorials/_data/XMM//heasarc-data.tar to /home/jovyan/projects/heasarc-tutorials/_data/XMM/ ... [astroquery.heasarc.core]
Data directory: /home/jovyan/projects/heasarc-tutorials/_data/XMM/

Setting SAS_ODF = /home/jovyan/projects/heasarc-tutorials/_data/XMM/0802710101/ODF

Running cifbuild with inputs: {} ...
Executing: 
cifbuild calindexset='ccf.cif' withccfpath='no' ccfpath='.' usecanonicalname='no' recurse='no' fileglob='*.ccf|*.CCF' fullpath='no' withobservationdate='no' observationdate='now' analysisdate='now' category='XMMCCF' ignorecategory='no' masterindex='no' withmasterindexset='no' masterindexset='ccf.mif' append='no'
cifbuild:- Exe

If you want more information on the function `basic_setup` run the cell below or see the long introduction tutorial.

In [11]:
obs.basic_setup?

Signature:
obs.basic_setup(
    data_dir=None,
    repo=None,
    overwrite=False,
    rerun=False,
    recalibrate=False,
    run_epproc=True,
    run_emproc=True,
    run_rgsproc=True,
    run_epchain=False,
    run_emchain=False,
    **kwargs,
)
Docstring:
Function to do all basic analysis tasks. The function will:

    1. Download data by calling 'download_data'
    2. Call the function 'calibrate_odf'
        A. Run 'cifbuild'
        B. Run 'odfingest'
    2. Run 'epproc' -OR- 'epchain'
    3. Run 'emproc' -OR- 'emchain'
    4. Run 'rgsproc'

If 'run_epchain' is set to 'True', then 'epproc' will not run.
If 'run_emchain' is set to 'True', then 'emproc' will not run.

Inputs:

    data_dir:    Data directory.
    repo:        Download repository ('esa','heasarc','fornax','aws').
    overwrite:   Remove previous data files and download again.
    rerun:       Rerun the *procs or *chains.
    recalibrate: Rerun 'cifbuild' and 'odfingest'.

All input arguments for 'download_ODF_data'

```{warning}
Note that this use of '?' is not valid in 'standard' Python, only in Juypyter notebooks and iPython.
```

## 2. Running SAS commands in Python

To run SAS tasks, especially ones not written in Python, you will need to use a wrapper from pySAS (MyTask). SAS tasks should be run from the work directory. The location of the work directory is stored as a variable in `obs.work_dir`.

### Moving to the working directory

In [12]:
start_dir = os.getcwd()

# Moving to the work directory
os.chdir(obs.work_dir)

### Executing your first pySAS task

The wrapper (which we imported as `w`) takes two inputs, the name of the SAS task to run, and a Python list of all the input arguments for that task. For example, to run a task with no input arguments you simply provide an empty list as the second argument.

The most common SAS tasks to run are: `epproc`, `emproc`, and `rgsproc`. Each one can be run without inputs (but some inputs are needed for more advanced analysis). These tasks have been folded into the function `basic_setup`, but they can be run individually.

In [13]:
inargs = []
MyTask("emproc", inargs).run()

Executing: 
emproc selectinstruments='no' emos1='no' emos2='no' removetemporaries='yes' removeintermediategtis='yes' removeintermediateeventlists='yes' withinstexpids='no' instexpids='M1S001 M2S001' selectccds='no' ccd1='no' ccd2='no' ccd3='no' ccd4='no' ccd5='no' ccd6='no' ccd7='no' selectmodes='yes' imaging='yes' rimaging='yes' timing='yes' ctiming='no' withgtiset='no' gtiset='gti.ds' runhkgtigen='yes' runatthkgen='yes' referencepointing='median' ra='0' dec='0' posangle='0' filterevents='yes' filterexpression='(FLAG & 0x762aa000) == 0' flagfilteredevents='no' rungtimerge='no' applygti='yes' runevlistcomb='yes' searchforbadpixels='yes' badpixfindalgo='embadpixfind' searchforbadcolumns='yes' withbadpixgti='no' badpixgti='bapixgti.ds' thresholdlabel='rate' lothresh='0' hithresh='0.005' columnsearchlabel='median' locolthresh='0' hicolthresh='0.002' flickertimesteps='1' flickerksthresh='0.55' flickerchisqthresh='15' backgroundrate='-1' narrowerthanpsf='3' threshabovebackground='no' loener

### Listing input arguments for pySAS tasks

You can list all input arguments available to any SAS task with option `'--help'` (or `'-h'`).

In [14]:
MyTask("emproc", ["-h"]).run()


        Usage: emproc [Options] param0=value0 [param1=value1] ...

        Options:
        -a | --ccfpath <dir1>:<dir2>...   Sets SAS_CCFPATH to <dir1>:<dir2>...
        -c | --noclobber                  Set SAS_CLOBBER=0
        -d | --dialog                     Launchs task GUI
        -f | --ccffiles <f1> <f2> ...     CCF files
        -h | --help                       Shows this message, display param file contents and exits
        -i | --ccf <cif>                  Sets SAS_CCF=<cif> (ccf.cif)
        -m | --manpage                    Opens web browser with task documentation
        -o | --odf <sumfile>              Sets SAS_ODF to SAS summary file
        -p | --param                      List of all available task parameters with default values
        -t | --trace                      Trace task execution
        -V | --verbosity <level>          Sets verbosity level and sets SAS_VERBOSITY
        -v | --version                    Shows task name and version, and exits
     

True

### Tasks with multiple input arguments

If there are multiple input arguments, then each needs to be a separate string in the Python list. For example, here is how to apply a "standard" filter. Our next pySAS call is equivalent to running the following 'standard' SAS command:

```
evselect table=unfiltered_event_list.fits withfilteredset=yes \
    expression='(PATTERN $<=$ 12)&&(PI in [200:12000])&&#XMMEA_EM' \
    filteredset=filtered_event_list.fits filtertype=expression keepfilteroutput=yes \
    updateexposure=yes filterexposure=yes
```

The input arguments should be in a list, with each input argument a separate string. Note: Some inputs require single quotes to be preserved in the string. This can be done using double quotes to form the string. i.e. `"expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'"`

In [15]:
unfiltered_event_list = "3278_0802710101_EMOS1_S001_ImagingEvts.ds"

inargs = [
    "table={0}".format(unfiltered_event_list),
    "withfilteredset=yes",
    "expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'",
    "filteredset=filtered_event_list.fits",
    "filtertype=expression",
    "keepfilteroutput=yes",
    "updateexposure=yes",
    "filterexposure=yes",
]

MyTask("evselect", inargs).run()

Executing: 
evselect table='3278_0802710101_EMOS1_S001_ImagingEvts.ds' keepfilteroutput='yes' withfilteredset='yes' filteredset='filtered_event_list.fits' destruct='yes' flagcolumn='EVFLAG' flagbit='-1' filtertype='expression' dssblock='' expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM' writedss='no' cleandss='no' updateexposure='yes' filterexposure='yes' blockstocopy='' attributestocopy='' energycolumn='PHA' withzcolumn='no' zcolumn='WEIGHT' withzerrorcolumn='no' zerrorcolumn='EWEIGHT' ignorelegallimits='no' withimageset='no' imageset='image.fits' xcolumn='RAWX' ycolumn='RAWY' imagebinning='imageSize' ximagebinsize='1' yimagebinsize='1' squarepixels='no' ximagesize='600' yimagesize='600' withxranges='no' ximagemin='1' ximagemax='640' withyranges='no' yimagemin='1' yimagemax='640' withimagedatatype='no' imagedatatype='Real64' withcelestialcenter='no' raimagecenter='0' decimagecenter='0' withspectrumset='no' spectrumset='spectrum.fits' spectralbinsize='5' withspecranges='no' 

## About this notebook

Author: Ryan Tanner, XMM GOF Scientist

Author: David J Turner, HEASARC Staff Scientist

Updated On: 2025-10-16

### Additional Resources

Support: [XMM Newton GOF Helpdesk](https://heasarc.gsfc.nasa.gov/docs/xmm/xmm_helpdesk.html)

### Acknowledgements

### References